# 层次聚类

这节我们要看的是层次聚类,它的目的就是形成一个层级的聚类情况,从下到上反映了从单点相粘合形成小聚类,小聚类形成大聚类的过程.不像K-means或者高斯混合模型这种划分聚类方法需要事先给一个聚类的数量K,一般它会形成一个树状图,允许我们在各个层次(对应不同数量的聚类)看聚类情况,从而克服了划分聚类方法由固定的K带来的缺点.

## 层次聚类分类

层次聚类方法按照构建层级的方法分为两种:

+ 聚合型 : 从下到上,从每个单点构成的聚类出发,一步步将相近的聚类粘合

+ 分裂型 : 从上到下,所有的点都属于一个聚类,慢慢将远的子集分裂出去形成新的聚类


## 度量和聚类距离

这个时候,我们就要好好定义下距离的"远""近"了.我们从点和点之间的[度量](./metric.ipynb)开始,到聚类和聚类之间的距离(linkage criterion).

基于点和点之间的度量$d$,我们就可以规定聚类A,B之间的距离.常用的有以下几种：
1. 最大距离（Complete-linkage clustering）,$\max\{ d(a,b) : a \in A, b\in B \}$

2. 最小距离(Single-linkage clustering),$\min\{ d(a,b) : a \in A, b\in B \}$

3. 平均距离(UPGMA),$ \frac {1} {n\times m} \sum \limits_ {a \in A, b\in B} d(a,b) \}$,其中$n,m$分别是A,B聚类的基数，也就是包含的点的数量.

4. 中点距离(UPGMC),$\vert d(C_A,C_B)  \vert$,其中$C_A, C_B$分别是A，B的中点，也就是离中心最近的点

5. 能量距离（Energy distance）,$ \frac {2} {n\times m} \sum \limits_ {a \in A, b\in B} \vert a-b\vert_2 - \frac{1}{n^2} \sum \limits_{a_i,a_j \in A} \vert a_i -a_j \vert_2 -\frac{1}{m^2} \sum \limits_{b_i,b_j \in B} \vert b_i -_j \vert_2 \}$

## SLINK算法介绍、复杂度

至于聚合型聚类的复杂度,不可能小于$ O(n^2) $,因为我们需要计算每对数据点之间的距离.我们介绍一种R.Sibson在1972年提出的[SLINK算法](https://grid.cs.gsu.edu/~wkim/index_files/papers/sibson.pdf),时间复杂度是$ O(n^2) $，空间复杂度是$ O(n) $.

我们把n个数据点从0到$ n-1 $编号，SLINK算法求出的是两个数组A和B，A(i)代表把i数据点和起码一个比i序号大的数据点并入一个聚类的层级(表现在树状图上是高度,本质上是聚类之间的距离),而B(i)表示这个层级包含i的聚类中序号最大的点.

得到这两个数组后

1. 我们可以从A中最小的数 $ a_0 $(可能有多个)开始,对应的点集$ \{i\}$ (因为可能多个,所以用集合表示)中的每个点,将它和它的B(i)划分为同一聚类并一直处于一个聚类,没有涉及的点单独成一个聚类;

2. 将$ a_0 $从A中去除，找A中剩下的最小的数,对应的点集$ \{i\} $中的每个点,将它和它的B(i)划入同一聚类并一直处于一个聚类,没有涉及的聚类(可能是单点聚类)保持不变.

3. 重复2步骤,直到把所有点划入一个聚类.


## 方法优缺点

相比于K-means方法，层次聚类每次运行都会得到相同结果，也可以在任何层次停止，处理的数据类型也更加丰富，比如可以通过设定合适的度量来处理字符串类型，但算法复杂度也会提高.

## ***使用sklearn中的接口实现层次聚类***

sklearn中有接口`sklearn.cluster.AgglomerativeClustering`可以实现一种叫做`凝聚层次聚类`的层次聚类算法.它通过递归地聚合样本实现层次聚类

另一类似的接口是`sklearn.cluster.FeatureAgglomeration`,它与`AgglomerativeClustering`不同之处在于它使用特征作为聚合的目标而非样本.

***例:将数据集[Wholesale+customers](http://archive.ics.uci.edu/ml/datasets/Wholesale+customers)聚类***

In [1]:
import requests
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import (
    adjusted_rand_score,
    adjusted_mutual_info_score,
    homogeneity_score,
    completeness_score,
    v_measure_score
)

### 数据获取

该数据集中的数据都是int类型,可以说可以直接使用

In [2]:
csv_content = requests.get("http://archive.ics.uci.edu/ml/machine-learning-databases/00292/Wholesale%20customers%20data.csv").text

In [3]:
csv_list = csv_content.strip().split("\n")

In [4]:
row_matrix = [line.strip().split(",") for line in csv_list]

In [5]:
row_name = row_matrix[0]

In [6]:
dataset = pd.DataFrame(row_matrix[1:],columns=row_name,dtype="int64")

In [7]:
dataset[:10]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
5,2,3,9413,8259,5126,666,1795,1451
6,2,3,12126,3199,6975,480,3140,545
7,2,3,7579,4956,9426,1669,3321,2566
8,1,3,5963,3648,6192,425,1716,750
9,2,3,6006,11093,18881,1159,7425,2098


In [8]:
dataset.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,1.322727,2.543182,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,0.468052,0.774272,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,1.000000,1.000000,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,1.000000,3.000000,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,2.000000,3.000000,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


### 数据预处理

可以看到虽然channel和region也是int类型,但明显和后面几个不同,channel和region应该是分类标签,后面的则是连续数据,而后面的数值变化跨度太大,因此可以对每个数据取对数作为特征.

In [9]:
for i in row_name[2:]:
    dataset[i] = np.log(dataset[i]+0.0000001)

In [10]:
dataset[:10]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,9.446913,9.175335,8.930759,5.365976,7.891331,7.198931
1,2,3,8.861775,9.191158,9.166179,7.474205,8.099554,7.482119
2,2,3,8.756682,9.083416,8.946896,7.785305,8.165079,8.967504
3,1,3,9.492884,7.086738,8.347827,8.764678,6.228511,7.488853
4,2,3,10.026369,8.596004,8.881558,8.272571,7.482682,8.553525
5,2,3,9.149847,9.019059,8.542081,6.501290,7.492760,7.280008
6,2,3,9.403107,8.070594,8.850088,6.173786,8.051978,6.300786
7,2,3,8.933137,8.508354,9.151227,7.419980,8.108021,7.850104
8,1,3,8.693329,8.201934,8.731013,6.052089,7.447751,6.620073
9,2,3,8.700514,9.314070,9.845911,7.055313,8.912608,7.648740


### 数据集拆分

In [11]:
train_set,validation_set = train_test_split(dataset)

### 训练模型

In [12]:
agc = AgglomerativeClustering(n_clusters=3)

In [13]:
agc.fit(train_set[row_name[2:]])

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward',
            memory=Memory(cachedir=None), n_clusters=3,
            pooling_func=<function mean at 0x0000000006278D08>)

### 模型评估

In [14]:
adjusted_rand_score(train_set["Channel"],agc.labels_)

0.68862283360500931

In [15]:
adjusted_mutual_info_score(train_set["Channel"],agc.labels_)

0.47921776460211823

In [16]:
homogeneity_score(train_set["Channel"],agc.labels_)

0.58996653662323317

In [17]:
completeness_score(train_set["Channel"],agc.labels_)

0.48133625813105552

In [18]:
v_measure_score(train_set["Channel"],agc.labels_)

0.53014383337979909